In [ ]:
#Installing pke

!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-x3ocfnbz
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-x3ocfnbz
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8763600 sha256=3c847c9961d82d83529500f270815a24a82742e98bd8daf7fe6188a7bc10309b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8uzgfu9l/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np 
import pandas as pd 
import os
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import re
import pke

In [ ]:
import os
from google.colab import drive 
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/NLP/Final')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
papers = pd.read_csv('201812_CL_Github.csv')

In [ ]:
papers.head()

,Abstract,Arxiv Link,Github Link,ID,Title
0,Discourse structures are beneficial for vari...,http://arxiv.org/abs/1812.00176,github.com/irit-melodi/irit-stac,1812.00176,A Deep Sequential Model for Discourse Parsing ...
1,Learning good representations is of crucial ...,http://arxiv.org/abs/1812.00271,github.com/mravanelli/SincNet/,1812.00271,Learning Speaker Representations with Mutual I...
2,The explosive growth in fake news and its er...,http://arxiv.org/abs/1812.00315,github.com/borisveytsman/acmart/issues/138,1812.00315,"Fake News: A Survey of Research, Detection Met..."
3,The amount of dialogue history to include in...,http://arxiv.org/abs/1812.00350,github.com/keras-team/keras,1812.00350,A Study on Dialogue Reward Prediction for Open...
4,Detecting controversy in general web pages i...,http://arxiv.org/abs/1812.00382,github.com/aboSamoor/polyglot,1812.00382,Improved and Robust Controversy Detection in G...


In [ ]:
papers.shape
#Total 106 papers given

(106, 5)

In [ ]:
ex = papers["Abstract"].iloc[3].replace("\n", " ")
ex

'  The amount of dialogue history to include in a conversational agent is often underestimated and/or set in an empirical and thus possibly naive way. This suggests that principled investigations into optimal context windows are urgently needed given that the amount of dialogue history and corresponding representations can play an important role in the overall performance of a conversational system. This paper studies the amount of history required by conversational agents for reliably predicting dialogue rewards. The task of dialogue reward prediction is chosen for investigating the effects of varying amounts of dialogue history and their impact on system performance. Experimental results using a dataset of 18K human-human dialogues report that lengthy dialogue histories of at least 10 sentences are preferred (25 sentences being the best in our experiments) over short ones, and that lengthy histories are useful for training dialogue reward predictors with strong positive correlations 

#### Keyphrase Extraction

In [ ]:
#TextRank code credit: https://www.kaggle.com/mohitr/keyphrase-extraction-from-research-papers

def extract_keyphrases(caption, n):
    extractor = pke.unsupervised.TextRank() 
    extractor.load_document(caption)
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=n, stemming=False)
    #print(keyphrases,"\n")
    return(keyphrases)
    
papers['Textrank_Keyphrases'] = papers.apply(lambda row: (extract_keyphrases(row['Abstract'],5)),axis=1)

In [ ]:
punc = '''!()[]{};:'"\,<>.?@#$^&*_~`…”“'''
corpus = []
corpus_raw = [a.replace("\n", " ") for a in papers["Abstract"]]
for a in corpus_raw:
  temp = ""
  for c in a:
    if not c in punc:
      temp += c.lower()
  if len(temp) > 0:
    corpus.append(temp)

print(len(corpus))

106


In [ ]:
stopw = stopwords.words('english')

def stopword_check(phrase):    #returns true if phrase contains no stopwords, false otherwise
  phrase = phrase.split(" ")
  for w in phrase:
    if w in stopw:
      return False
  return True
  

def get_vocab(abstract):
  my_words = nltk.word_tokenize(abstract)
  my_bigrams = nltk.bigrams(my_words)
  my_trigrams = nltk.trigrams(my_words)
  words = [(w,) for w in my_words]
  bigrams = [b for b in my_bigrams]
  trigrams = [t for t in my_trigrams]
  vocab = words + bigrams + trigrams
  vocab = set(vocab)                       #list of tuples
  vocab = [list(v) for v in vocab]         #list of lists
  vocab = [" ".join(v) for v in vocab]     #list of strings
  vocab_cleaned = []
  for v in vocab:
    if stopword_check(v):
      vocab_cleaned.append(v)
  return vocab_cleaned

In [ ]:
def take_best(scores, n):
  a = sorted(scores, key=scores.get, reverse=True)[:n]
  return [(b, scores[b]) for b in a]

#tf-idf
tfidf = []
for abstract in corpus:
  scores = dict()
  vocab = get_vocab(abstract)
  for v in vocab:
    term_count = abstract.count(v)
    doc_count = 0
    for c in corpus:
      if v in c:
        doc_count += 1
    if doc_count == 0:
      doc_count = 1
    scores[v] = (term_count/len(abstract.split(" "))) * np.log(len(corpus)/doc_count)
  tfidf.append(take_best(scores, 5))
papers['Tf-idf_Keyphrases'] = tfidf

In [ ]:
f = open("nlp_keywords.txt", "r")
manual = f.read().splitlines()[:106]
f.close()
manual = [m.split(", ") for m in manual]
papers['Manual_Keyphrases'] = manual
papers

,Abstract,Arxiv Link,Github Link,ID,Title,Textrank_Keyphrases,Tf-idf_Keyphrases,Manual_Keyphrases
0,Discourse structures are beneficial for vari...,http://arxiv.org/abs/1812.00176,github.com/irit-melodi/irit-stac,1812.00176,A Deep Sequential Model for Discourse Parsing ...,"[(discourse dependency structures, 0.130893405...","[(discourse, 0.16946367923698083), (discourse ...","[discourse structures, deep sequential model, ..."
1,Learning good representations is of crucial ...,http://arxiv.org/abs/1812.00271,github.com/mravanelli/SincNet/,1812.00271,Learning Speaker Representations with Mutual I...,"[(useful speaker representations, 0.0751563194...","[(mutual information, 0.07248869058205286), (m...","[sincnet architecture, mutual information, hig..."
2,The explosive growth in fake news and its er...,http://arxiv.org/abs/1812.00315,github.com/borisveytsman/acmart/issues/138,1812.00315,"Fake News: A Survey of Research, Detection Met...","[(current fake news research, 0.16925477623337...","[(fake news, 0.236123498436054), (fake, 0.2361...","[fake news, interdisciplinary research, propag..."
3,The amount of dialogue history to include in...,http://arxiv.org/abs/1812.00350,github.com/keras-team/keras,1812.00350,A Study on Dialogue Reward Prediction for Open...,"[(dialogue reward predictors, 0.15061766476375...","[(dialogue, 0.13620311544911662), (dialogue re...","[dialogue history, conversational agent, optim..."
4,Detecting controversy in general web pages i...,http://arxiv.org/abs/1812.00382,github.com/aboSamoor/polyglot,1812.00382,Improved and Robust Controversy Detection in G...,"[(cross - domain performance, 0.10065964960247...","[(controversy, 0.1589808782083659), (general w...","[detecting controversy, problematic content, n..."
...,...,...,...,...,...,...,...,...
101,We propose the first joint model for Vietnam...,http://arxiv.org/abs/1812.11459,github.com/acl-org/acl-pub/issues/2,1812.11459,A neural joint model for Vietnamese word segme...,"[(joint model obtains state, 0.164290755490313...","[(vietnamese, 0.15289964242990384), (tagging, ...","[joint model, vietnamese, word segmentation, v..."
102,This paper proposes a variational self-atten...,http://arxiv.org/abs/1812.11559,github.com/FakeNewsChallenge/fnc-1,1812.11559,Variational Self-attention Model for Sentence ...,"[(multi - modal attention distributions, 0.236...","[(vsam, 0.15206866611235), (self-attention, 0....","[self-attention, variational inference, attent..."
103,While the volume of scholarly publications h...,http://arxiv.org/abs/1812.11709,github.com/GraphEmbedding/HRLHG\label{foot:web,1812.11709,Cross-language Citation Recommendation via Hie...,[(cross - language citation recommendation tas...,"[(publication, 0.08314747462936381), (cross-la...","[useful candidate papers, foreign language rep..."
104,The correct interpretation of quantifier sta...,http://arxiv.org/abs/1812.11737,github.com/goodfeli/dlbook_notation.,1812.11737,"The meaning of ""most"" for visual question answ...","[(visual question answering model, 0.173924005...","[(scene, 0.08555584333065498), (visual questio...","[quantifier statements, inference mechanisms, ..."


In [ ]:
#keyphrase lengths
N = 5*106    #total number of keyphrases
avglen_manual = sum([len(phrase.split(" ")) for keywords in papers["Manual_Keyphrases"] for phrase in keywords])/N
avglen_textrank = sum([len(phrase[0].split(" ")) for keywords in papers["Textrank_Keyphrases"] for phrase in keywords])/N
avglen_tfidf = sum([len(phrase[0].split(" ")) for keywords in papers["Tf-idf_Keyphrases"] for phrase in keywords])/N
print("Average keyphrase lengths:", avglen_textrank, avglen_tfidf, avglen_manual)

Average keyphrase lengths: 3.1320754716981134 1.3773584905660377 1.9528301886792452


In [ ]:
#fuzzy accuracy evaluation
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz

In [ ]:
#helper functions for code cleanliness

def get_textrank(i,j):
  global papers
  return papers['Textrank_Keyphrases'].iloc[i][j][0]

def get_tfidf(i,j):
  global papers
  return papers['Tf-idf_Keyphrases'].iloc[i][j][0]

def get_manual(i,j):
  global papers
  return papers['Manual_Keyphrases'].iloc[i][j]

In [ ]:
def evaluate():
  correct_textrank = 0
  correct_tfidf = 0
  correct_textrank_partial = 0
  correct_tfidf_partial = 0
  x = 0
  for i in range(106):
    correct_textrank += max([fuzz.ratio(get_textrank(i,j), get_manual(i,j)) for j in range(5)])
    correct_tfidf += max([fuzz.ratio(get_tfidf(i,j), get_manual(i,j)) for j in range(5)])
    correct_textrank_partial += max([fuzz.partial_ratio(get_textrank(i,j), get_manual(i,j)) for j in range(5)])
    correct_tfidf_partial += max([fuzz.partial_ratio(get_tfidf(i,j), get_manual(i,j)) for j in range(5)])
  return [correct_textrank/106, correct_tfidf/106, correct_textrank_partial/106, correct_tfidf_partial/106]

In [ ]:
evaluate()

[61.528301886792455, 62.990566037735846, 75.06603773584905, 82.83018867924528]